In [20]:
import os
from builtins import int

import pandas as pd

data = pd.read_csv("data/주요재무지표.csv", encoding="euc-kr")
# 분석에 사용하지 않을 컬럼 드랍
data.drop(["자산총계", "부채총계", "자본총계", "매출액", "영업이익", "당기순이익"], axis=1, inplace=True)

In [21]:
# settle_month_dict -> 기업별 결산 월

stock_info_data = pd.read_csv("data/종목정보.txt", sep="\t", encoding="euc-kr")
settle_month_dict = stock_info_data.set_index("Name")["SettleMonth"]
settle_month_dict = settle_month_dict.apply(lambda x:int(x[:-1])).to_dict()

In [22]:
# sub_due_dict -> {[결산월, 사업연도] : 제출마감일}

sub_due_data = pd.read_csv("data/사업보고서_제출마감일.csv", encoding="euc-kr")
sub_due_dict = sub_due_data.set_index(["결산 월", "사업연도"])["제출마감일"].to_dict()

In [23]:
# sp_data_dict -> {종목명 : 주가 데이터}
import os
from tqdm import tqdm

sp_data_dict = dict()
for stock in tqdm(data["기업"].unique()):
    if stock + ".csv" in os.listdir("data/주가데이터"):
        sp_data = pd.read_csv("data/주가데이터/" + stock + ".csv", usecols=["Date", "Close"], parse_dates=["Date"])
        sp_data_dict[stock] = sp_data

100%|██████████| 2370/2370 [00:19<00:00, 118.88it/s]


In [24]:
# 데이터 병합
# 주가 데이터와 날짜가 입력됐을 때, 해당 날짜를 포함해서 가장 가까운 미래의 종가를 반환하는 함수
import numpy as np


def find_closest_stock_price(sp_data, date):
    date = pd.to_datetime(date)
    # 주가 데이터를 벗어나면 결측으로 반환
    if sp_data["Date"].max() < date:
        return np.nan
    else:
        while True:
            if sum(sp_data["Date"] == date) > 0:
                value = sp_data.loc[sp_data["Date"] == date, "Close"].iloc[0]
                break
            else:
                date += pd.to_timedelta(1, 'D')

        return value



In [25]:
cur_sp_list = []  # 현재 주가
next_sp_list = []  # 미래 주가

for corp, year in tqdm(data[["기업", "연도"]].values):
    if corp not in sp_data_dict.keys():
        cur_sp_list.append(np.nan)
        next_sp_list.append(np.nan)
    else:
        sp_data = sp_data_dict[corp]
        settle_month = settle_month_dict[corp]

        # 현재 주가 탐색 및 추가
        try:
            cur_date = sub_due_dict[settle_month, year]
            cur_sp = find_closest_stock_price(sp_data, cur_date)
            cur_sp_list.append(cur_sp)
        except:
            cur_sp_list.append(np.nan)

        # 미래 주가 탐색 및 추가
        try:
            next_date = sub_due_dict[settle_month, year+1]
            next_sp = find_closest_stock_price(sp_data, next_date)
            next_sp_list.append(next_sp)
        except:
            next_sp_list.append(np.nan)


data["현재_주가"] = cur_sp_list
data["미래_주가"] = next_sp_list

100%|██████████| 18044/18044 [06:40<00:00, 45.08it/s] 


In [26]:
# 배당금 및 PER 데이터 병합

div_data = pd.read_csv("data/주당배당금.csv", encoding="euc-kr")
PER_data = pd.read_csv("data/PER.csv", encoding="euc-kr")

In [27]:
print(div_data.head())
print(PER_data.head())

  stock_name   2013   2014   2015   2016   2017   2018   2019   2020
0         3S    NaN    NaN    NaN    0.0    0.0    0.0    0.0    0.0
1     AJ네트웍스    0.0    0.0    0.0   60.0   86.0  100.0  300.0  210.0
2      AK홀딩스  200.0  350.0  500.0  550.0  650.0  750.0  750.0  400.0
3     APS홀딩스    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0
4      AP시스템    NaN    NaN    NaN    0.0    0.0  150.0   50.0  120.0
      2013        2014   2015        2016        2017        2018        2019  \
0      NaN         NaN    NaN         NaN         NaN         NaN         NaN   
1      NaN         NaN    NaN   96.666667   83.372093   48.100000    9.833333   
2  278.655  281.428571  112.2  111.272727  114.769231   69.866667   24.000000   
3      NaN         NaN    NaN         NaN         NaN         NaN         NaN   
4      NaN         NaN    NaN         NaN         NaN  192.000000  490.000000   

     2020 stock_name  
0     NaN         3S  
1   22.00     AJ네트웍스  
2   67.25      AK홀딩스  
3     N

In [28]:
# 데이터 구조 변환
# div와 PER 데이터가 하나의 행이 한 기업과 사업 연도의 배당금과 PER을 나타내도록 변환

year_col_list = ["2013","2014","2015","2016","2017","2018","2019","2020"]
div_data = div_data.melt(id_vars=["stock_name"], value_vars=year_col_list)

In [29]:
div_data.head()

,stock_name,variable,value
0,3S,2013,NaN
1,AJ네트웍스,2013,0.0
2,AK홀딩스,2013,200.0
3,APS홀딩스,2013,0.0
4,AP시스템,2013,NaN


In [30]:
div_data.rename({"stock_name":"기업", "variable":"연도", "value":"배당금"}, axis=1, inplace=True)
div_data["연도"] = div_data["연도"].astype(int)

In [31]:
PER_data = PER_data.melt(id_vars=["stock_name"], value_vars=year_col_list)
PER_data.rename({"stock_name":"기업", "variable":"연도", "value":"PER"}, axis=1, inplace=True)
PER_data["연도"] = PER_data["연도"].astype(int)

In [32]:
data = pd.merge(data, div_data, on=["기업", "연도"], how="left")
data = pd.merge(data, PER_data, on=["기업", "연도"], how="left")

In [33]:
data.head()

,기업,연도,부채비율,매출액증가율,영업이익증가율,당기순이익증가율,매출액_상태,영업이익_상태,당기순이익_상태,ROA,ROE,현재_주가,미래_주가,배당금,PER
0,3S,2013,70.612320,NaN,NaN,NaN,적자전환,적자전환,적자전환,1.036271,NaN,6100.0,4350.0,NaN,NaN
1,3S,2014,60.288336,-11.687845,173.179128,114.728018,훅자지속,적자전환,적자전환,-6.675781,-10.489496,4350.0,2535.0,NaN,NaN
2,3S,2015,56.323967,-24.352916,38.894997,14.974005,훅자지속,적자지속,적자지속,-8.312107,-13.836280,2535.0,3595.0,NaN,NaN
3,3S,2016,59.923676,2.240210,453.409466,-64.511098,훅자지속,훅자전환,적자지속,-5.893767,-9.383155,3595.0,2320.0,0.0,NaN
4,3S,2017,54.331315,10.359573,148.592592,61.341308,훅자지속,적자전환,적자지속,-16.392325,-27.605322,2320.0,2685.0,0.0,NaN


In [34]:
data.to_csv("data/재무제표_분석용데이터.csv", index=False, encoding="euc-kr")